In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install groq
!pip install openai
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-1.7B", device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B", trust_remote_code=True)

# Load LoRA adapter vào base model
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/SICT/qwen_SICT")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
import gradio as gr
import os
from openai import OpenAI
import torch

# === Thiết lập API Groq ===
os.environ["GROQ_API_KEY"] = "apikey_you"
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

# === Hàm xử lý câu hỏi ===
def get_answer(question, choices_raw):
    try:
        choices = choices_raw.strip().split("\n")
        choices_text = "\n".join(choices)

        # Prompt cho mô hình
        prompt = f"""<|im_start|>system
Bạn là một chuyên gia về trí tuệ nhân tạo. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn (nếu câu hỏi đáp án dưới dạng LaTeX thì hãy dịch sang tiếng Việt rồi giải), chọn đáp án đúng trong các đáp án trên và giải thích chi tiết.
<|im_start|>user
### Câu hỏi: {question}
### Các lựa chọn: \n{choices_text}
### Câu trả lời:
<|im_start|>assistant
"""

        # Gửi prompt đến Groq
        response = client.chat.completions.create(
            model="deepseek-llama3-8b",  # Thay đổi model nếu cần
            messages=[
                {"role": "system", "content": "Bạn là một chuyên gia AI, hãy giải thích và đưa đáp án"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=512
        )

        text = response.choices[0].message.content
        if "</think>" in text:
            return text.split("</think>", 1)[1].strip()
        else:
            return text.strip()
    except Exception as e:
        return f"Đã xảy ra lỗi: {str(e)}"

# === Giao diện Gradio ===
iface = gr.Interface(
    fn=get_answer,
    inputs=[
        gr.Textbox(label="Câu hỏi trắc nghiệm", lines=3, placeholder="Ví dụ: AI là viết tắt của từ nào?"),
        gr.Textbox(label="Các lựa chọn (mỗi dòng 1 lựa chọn)", lines=5, placeholder="A. Artificial Intelligence\nB. Automatic Interface\n...")
    ],
    outputs=gr.Textbox(label="Đáp án và giải thích"),
    title="Giải thích câu hỏi trắc nghiệm AI bằng DeepSeek qua Groq",
    description="Sử dụng Groq API và mô hình DeepSeek để giải thích câu hỏi trắc nghiệm.",
)

# === Chạy giao diện (phải dùng share=True để có link trên Kaggle) ===
iface.launch(share=True, inline=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e0ab396223b92ed7bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
